# Transfer Learning with TensorFlow Part 2: Fine-Tuning

> In the previous notebook, we covered transfer learning feature extraction. Now it's time to learn about a new kind of transfer learning: `Fine Tuning`.

In [1]:
# Check if we use the TPU
!nvidia-smi

Wed Jan 24 19:19:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Creating helpher functions

> In previous notebooks, we have created a bunch of helper functions, now we could rewrite them all, however, this is tedious.

So, it's a good idea to put functions you'll want to use again in a script you can download and import into your notebooks or elsewhere.

We have done this for some of the functions we have used previously here:

https://raw.githubusercontent.com/oguzkaganbilici/TensorFlow-Developer-Certificate-in-2023-Zero-to-Mastery/main/helper_functions.py


In [2]:
!wget https://raw.githubusercontent.com/oguzkaganbilici/TensorFlow-Developer-Certificate-in-2023-Zero-to-Mastery/main/helper_functions.py

--2024-01-24 19:19:52--  https://raw.githubusercontent.com/oguzkaganbilici/TensorFlow-Developer-Certificate-in-2023-Zero-to-Mastery/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2665 (2.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   2.60K  --.-KB/s    in 0s      

2024-01-24 19:19:52 (41.3 MB/s) - ‘helper_functions.py’ saved [2665/2665]



In [5]:
# Import helper functions we're going to use in this notebook

from helper_functions import create_tensorboard_callback, unzip_file
from helper_functions import plot_loss_curves, walk_in_dir

## Let's get some data

> This time we're going to see how we can use the pre-trained models within `tf.keras.application` and apply them to our own problem (recognizing images of food)

In [6]:
# Get 10% of training data of 10 classes of Food101

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_file("10_food_classes_10_percent.zip")

--2024-01-24 19:28:29--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 142.250.107.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   163MB/s    in 1.0s    

2024-01-24 19:28:30 (163 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



1

In [8]:
# Check out how many images and subdirectories are in our dataset

walk_in_dir("/content/10_food_classes_10_percent")

There are 2 of directories and 0 images in /content/10_food_classes_10_percent.
There are 10 of directories and 0 images in /content/10_food_classes_10_percent/test.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/hamburger.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/grilled_salmon.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/ramen.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/sushi.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/chicken_wings.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/pizza.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/steak.
There are 0 of directories and 250 images in /content/10_food_classes_10_percent/test/ice_cream.
There are 0 of directories and 250 images in /content/10_food_cla

In [9]:
# Creating training and testing dir

train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = image_dataset_from_directory(directory=train_dir,
                                                     image_size=IMG_SIZE,
                                                     label_mode="categorical",
                                                     batch_size=BATCH_SIZE,
                                                     )

test_data = image_dataset_from_directory(directory=test_dir,
                                         image_size=IMG_SIZE,
                                         label_mode="categorical",
                                         batch_size=BATCH_SIZE)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [14]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

* Create batches of 32 images of size 224x224 split into red, green, blue color channels.

In [15]:
# Check out the clas names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']